In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
import cv2
import os

# Define paths to image folders
train_low_folder = "C:/Users/scdes/OneDrive/Desktop/archive/lol_dataset/Train/low"  # Path to folder containing low-resolution training images
train_high_folder = "C:/Users/scdes/OneDrive/Desktop/archive/lol_dataset/Train/high"  # Path to folder containing high-resolution training images
eval_low_folder = "C:/Users/scdes/OneDrive/Desktop/archive/lol_dataset/Test/low"  # Path to folder containing low-resolution evaluation images
eval_high_folder = "C:/Users/scdes/OneDrive/Desktop/archive/lol_dataset/Test/high"  # Path to folder containing high-resolution evaluation images

# Load low-resolution and high-resolution images using OpenCV
train_low_images = [cv2.imread(os.path.join(train_low_folder, filename)) for filename in os.listdir(train_low_folder)]
train_high_images = [cv2.imread(os.path.join(train_high_folder, filename)) for filename in os.listdir(train_high_folder)]
eval_low_images = [cv2.imread(os.path.join(eval_low_folder, filename)) for filename in os.listdir(eval_low_folder)]
eval_high_images = [cv2.imread(os.path.join(eval_high_folder, filename)) for filename in os.listdir(eval_high_folder)]

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Define the dataset class
class CustomDataset(Dataset):
    def __init__(self, low_res_images, high_res_images, transform=None):
        self.low_res_images = low_res_images
        self.high_res_images = high_res_images
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, idx):
        low_res_image = self.low_res_images[idx]
        high_res_image = self.high_res_images[idx]

        if self.transform:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image.to(device), high_res_image.to(device)

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Create custom datasets
train_dataset = CustomDataset(train_low_images, train_high_images, transform=transform)
eval_dataset = CustomDataset(eval_low_images, eval_high_images, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=1)

In [3]:
# Define the model architecture
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.batchnorm = nn.BatchNorm2d(channels)
        self.activation = nn.ReLU()

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.batchnorm(x)
        x = self.activation(x)
        x = self.conv1(x)
        x = self.batchnorm(x)
        x += residual
        x = self.activation(x)
        return x

class DiffusionModel(nn.Module):
    def __init__(self):
        super(DiffusionModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.activation = nn.ReLU()
        self.residual_blocks = nn.ModuleList([ResidualBlock(64) for _ in range(8)])
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.activation(x)
        for block in self.residual_blocks:
            x = block(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.activation(x)
        x = self.conv3(x)
        x = self.sigmoid(x)
        return x

In [4]:
# Initialize the model and move it to the device
model = DiffusionModel().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(50):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    print(f"Epoch {epoch+1}, Training Loss: {running_loss / len(train_dataset)}")

Epoch 1, Training Loss: 0.16694472879555422
Epoch 2, Training Loss: 0.16026146386164367
Epoch 3, Training Loss: 0.1586519350364958
Epoch 4, Training Loss: 0.15794278440478537
Epoch 5, Training Loss: 0.15625109654880062
Epoch 6, Training Loss: 0.15586176884466224
Epoch 7, Training Loss: 0.1548072091198152
Epoch 8, Training Loss: 0.15365665394851227
Epoch 9, Training Loss: 0.15312550265879668
Epoch 10, Training Loss: 0.15289105550798865
Epoch 11, Training Loss: 0.15281893204052732
Epoch 12, Training Loss: 0.15223993650762382
Epoch 13, Training Loss: 0.15241134468581258
Epoch 14, Training Loss: 0.1517606733578075
Epoch 15, Training Loss: 0.1520092365015106
Epoch 16, Training Loss: 0.15077993794507588
Epoch 17, Training Loss: 0.15091620184020282
Epoch 18, Training Loss: 0.15067109908985415
Epoch 19, Training Loss: 0.1504421970739807
Epoch 20, Training Loss: 0.14995513027437876
Epoch 21, Training Loss: 0.15029869561573278
Epoch 22, Training Loss: 0.14992297512776764
Epoch 23, Training Loss:

In [ ]:
# Evaluate the model
model.eval()
eval_loss = 0.0
with torch.no_grad():
    for inputs, targets in eval_loader:
        outputs = model(inputs)
        eval